# NOTEBOOK PARA OBTENER LOS HITS Y MEJORES HITS
Esto incluye la limpieza que se requiere ya que se tienen dobles espacios en cada línea.

**Que pasos debo seguir para elegir todos los raw hits de acuerdo a lo indicado por Salva:**

"...En realidad, hablando con mi hermano hemos decidido que los selected pulses no cal guardarlos, los usas para el match, y coges todos los hits que coincidan (cada uno de ellos será una nueva fila en tu CVS). Como decía antes, en las columnas tendrás toda la información del muón, del bbfit, del affit, y del hit coincidente con un selected pulse particular."

El pseudocódigo que se me ocurre es el siguiente:

1. Seleccionar el primer "selected pulse" de la lista de seleccionados.
2. Filtrar el df de raw hits por todos los que coincidan en linea detector y piso detector.
3. Filtrar el resultado viendo cuales coinciden en los parámetrox X', Y', Z' y X, Y, Z.
4. Guardar el resultado en el dataframe final junto con runID, frameID
5. Iterar hasta que ya no haya mas selected pulses
6. Hacer lo mismo para cada evento del archivo seleccionado

In [1]:
#Primero algunas variables importantes de ubicación de los archivos y librerías
import os
import pandas as pd
import re
import math

#extractpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este es para mi no tan nueva PC-Desktop
#csv_path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\" #Path para mi PC Desktop
#csv_path = "F:\\DATA_TFM\\CSVs\\" #Path para mi PC en el trabajo
#extractpath = "F:\\DATA_TFM\\TXTs" #Este path es para mi PC en el trabajo
extractpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este path es para mi laptop en casa
csv_path = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\" #Path para mi latptop en casa

txtfilesarr = os.listdir(extractpath); txtfilesarr = sorted(txtfilesarr)
csvarr = os.listdir(csv_path); csvarr = sorted(csvarr)

In [2]:
# Funcion para obtener los indices de un archivo
def indices(df):
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = [];#ind_aafit = []
    for i in range(data.shape[0]):
        tempstr = data[0][i]
        if "start_event" in tempstr: ind_startev.append(i)
        if "UTC" in tempstr: ind_runid.append(i)
        if "weights" in tempstr: ind_weights.append(i)
        if "muon" in tempstr: ind_muon.append(i)
        #if "aafit" in tempstr:  ##NO LO CALCULARE!!! OBTENDRE LOS VALORES DE OTRA FORMA
        #    ind_aafit.append(i) ##COMO TENGO EL INDICE DE BBFIT, RESTO 1 LINEA Y ASIGNO VALORES
        if "bbfit" in tempstr:   ##SEGUN LA EXISTENCIA O NO DE LA LINEA!!!! EUREKA!!!
            ind_bbfit.append(i)
        if "selected" in tempstr: ind_bb_selp.append(i)
        if "end_event" in tempstr: ind_endev.append(i)
    return(ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev)

#Función para limpiar un hit (lista) de elementos tipo dobles espacios y simples
def limpiahit(templist):
    cleanlist = []
    for j in range(len(templist)):
        if templist[j]!="":
            cleanlist.append(templist[j])
    return(cleanlist)



In [3]:
###################################
### OBTENIENDO LOS MEJORES HITS ###
###################################

## Por el momento será para un solo archivo, posteriormente haré el bucle.

#Leyendo el archivo csv ya sin las líneas no deseadas en un dataframe
file = csv_path+csvarr[0]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None)

#Obteniendo los indices necesarios
ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

El archivo trabajando es:  U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_anumu_CC_a_reco.i3.csv


In [4]:
##### Código para obtener los raw hits y los selected pulses, en forma de listas.

#Encontramos primero el # de lineas de rawhits y selected pulses
difrawhits = ind_bb_selp[1] - ind_bbfit[1] #debe variar con la variable del bucle de eventos
difselhits = ind_endev[1] - ind_bb_selp[1] #debe variar con la variable del bucle de eventos

start_rawhit = ind_bbfit[1]+1; #debe variar con el bucle de eventos
start_selhit = ind_bb_selp[1]+1 #debe variar con el bucle de eventos

raw_hits = []; hits_raw_limpios = []
sel_hits = []; sel_hits_limpios = []

#Obteniendo solo los raw_hits de un evento
for i in range(difrawhits-1):
    temprawhit = list(data[0][start_rawhit+i].split(" "))
    raw_hits.append(temprawhit)
#Limpiamos los raw hits obtenidos del evento
for i in range(len(raw_hits)):
    hits_raw_limpios.append(limpiahit(raw_hits[i]))

#Obteniendo solo los selected pulses (hits) del mismo evento
for i in range(difselhits-1):
    tempselhit = list(data[0][start_selhit+i].split(" "))
    sel_hits.append(tempselhit)
#Limpiamos los raw hits obtenidos del evento
for i in range(len(sel_hits)):
    sel_hits_limpios.append(limpiahit(sel_hits[i]))

In [5]:
## Verificación para saber si obtuve realmente los raw hits y los selected pulses.
print("RAW HITS LIMPIOS")
print(len(raw_hits),hits_raw_limpios)
print("\nSELECTED HITS LIMPIOS")
print(len(sel_hits_limpios),sel_hits_limpios)

RAW HITS LIMPIOS
138 [['hit', '1', '3', '2', '0', '-9.595928', '36.727814', '-156.955892', '0.461464', '-0.535772', '-0.707106', '-49.247006', '0.735974', '429.115295'], ['hit', '2', '3', '2', '0', '-9.595928', '36.727814', '-156.955892', '0.461464', '-0.535772', '-0.707106', '2779.497053', '0.853491', '429.115295'], ['hit', '3', '3', '2', '0', '-9.595928', '36.727814', '-156.955892', '0.461464', '-0.535772', '-0.707106', '1748.278818', '1.412297', '429.115295'], ['hit', '4', '3', '2', '2', '-10.540966', '37.058107', '-156.955892', '-0.694724', '-0.131753', '-0.707106', '2646.821118', '1.234366', '222.778320'], ['hit', '5', '4', '17', '0', '44.562730', '9.519969', '60.984108', '-0.493182', '0.506726', '-0.707106', '-456.936516', '0.580498', '63.953400'], ['hit', '6', '4', '17', '2', '45.526100', '9.247741', '60.984108', '0.685429', '0.173745', '-0.707106', '-1803.419144', '1.130774', '374.145508'], ['hit', '7', '4', '17', '2', '45.526100', '9.247741', '60.984108', '0.685429', '0.173745

In [6]:
#Código para analizar la correlación entre selected pulses y raw hits y asignar el módulo óptico.

#Idea... pasemos a dataframe las listas que tenemos
columnas = ['hit','#hit','lin_det','piso_det','mód_opt',
            'X\'','Y\'','Z\'','X','Y','Z','tiempo','amp','frec_hits']
raw_hits_ev_df = pd.DataFrame(hits_raw_limpios,columns=columnas)
sel_hits_ev_df = pd.DataFrame(sel_hits_limpios,columns=columnas)


#Tomo como base el primer elemento de los selected pulses
#print(sel_hits_limpios[0])

#Debo comparar contra todos y cada uno de los raw hits
#Sin embargo esto es fuerza bruta y no se si exista alguna mejor forma... como buscar por pisos primero


In [7]:
#raw_hits_ev_df.sample(10)
sel_hits_ev_df

,hit,#hit,lin_det,piso_det,mód_opt,X',Y',Z',X,Y,Z,tiempo,amp,frec_hits
0,hit,1,5,12,0,-58.891665,77.891023,-11.706892,-0.622119,-0.336108,-0.707106,302.447963,0.610659,72.975159
1,hit,2,5,13,0,-58.955763,78.087135,2.815108,-0.700534,-0.096191,-0.707106,315.962909,0.742721,55.351257
2,hit,3,5,14,0,-58.843884,77.816772,17.343108,-0.563666,-0.426944,-0.707106,340.341512,7.562943,46.081543
3,hit,4,5,15,0,-57.844178,77.956935,31.872108,0.659343,-0.255474,-0.707106,368.401895,3.664037,63.133240
4,hit,5,5,16,0,-58.912526,78.397776,46.383108,-0.647639,0.283838,-0.707106,417.011841,4.868736,0.000000
5,hit,6,5,17,0,-57.810655,78.245451,60.932108,0.700355,0.097488,-0.707106,470.932161,7.625308,55.103302
6,hit,7,5,18,0,-57.909454,77.834539,75.454108,0.579488,-0.405209,-0.707106,536.688133,3.934456,59.719086
7,hit,8,5,20,0,-58.443406,78.740612,104.507108,-0.073733,0.703252,-0.707106,663.952405,3.977190,86.517334


In [8]:
#### FILTRO PARA BUSCAR LOS HITS QUE HACEN MATCH CON EL SELECTED PULSE Y QUEDARNOS 
#### CON LA DATA DE MODULO OPTICO (ENTIENDASE RENGLON O RENGLONES QUE CORRESPONDEN AL SELECTED HIT)
#### EL CRITERIO ES QUEDARSE CON TODOS LOS RAW HITS QUE SEAN IGUALES EN X',Y',Z',X,Y, y Z.

#Abro el archivo a trabajar y paso el contenido a un dataframe
file = csv_path+csvarr[0]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None)

#Obteniendo los indices necesarios
#ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

#Variables auxiliares
columnas = ['hit','#hit','lin_det','piso_det','mód_opt',
            'X\'','Y\'','Z\'','X','Y','Z','tiempo','amp','frec_hits']
df_hits = pd.DataFrame(columns=columnas)
#Obtengo el runID que es el mismo para cada archivo
runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])
#Obtengo el frameID que es diferente para cada evento
temprunid = list(data[0][ind_runid[1]].split(" ")) #Esto va en el bucle para recorrer todos los archivos
frameID = temprunid[1]; trigger_counter = temprunid [2]
print("runID:",runID,"frameID:",frameID)

#Este bucle recorre cada row de los selected pulses
for j in range(sel_hits_ev_df.shape[0]):
    selp = sel_hits_ev_df.iloc[j][5:11]
    #Este bucle recorre cada row de los raw hits seleccionados.
    lindet_search = sel_hits_ev_df.iloc[j][2] #Linea detector a buscar
    storey_search = sel_hits_ev_df.iloc[j][3] #Piso detector a buscar
    #Primero busqué por piso detector (me pareció que era más corto ya que son menos lineas)
    tempdf = raw_hits_ev_df.loc[raw_hits_ev_df['piso_det'].isin([storey_search])]
    #Posteriormente filtré por línea del detector
    tempdf = tempdf.loc[tempdf['lin_det'].isin([lindet_search])]
    #print(tempdf)
    for i in range(tempdf.shape[0]):
        temp = tempdf.iloc[i][5:11]
        if selp.equals(temp):
            df_hits=df_hits.append(tempdf.iloc[i])
        #print("\n")

El archivo trabajando es:  U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_anumu_CC_a_reco.i3.csv
runID: 25800 frameID: 66099


In [9]:
#Verificando lo que obtuvo el algoritmo de la celda anterior, que en teoría son todos los match
df_hits.sort_values('piso_det')
#Falta agregar las columnas de runID y frameID para despues poder hacer el join con la data del muon, aafit y bbfit

,hit,#hit,lin_det,piso_det,mód_opt,X',Y',Z',X,Y,Z,tiempo,amp,frec_hits
25,hit,26,5,12,0,-58.891665,77.891023,-11.706892,-0.622119,-0.336108,-0.707106,302.447963,0.610659,72.975159
92,hit,93,5,14,0,-58.843884,77.816772,17.343108,-0.563666,-0.426944,-0.707106,340.341512,1.267171,46.081543
93,hit,94,5,14,0,-58.843884,77.816772,17.343108,-0.563666,-0.426944,-0.707106,698.750958,2.270429,46.081543
96,hit,97,5,15,0,-57.844178,77.956935,31.872108,0.659343,-0.255474,-0.707106,1422.256735,0.659200,63.133240
109,hit,110,5,17,0,-57.810655,78.245451,60.932108,0.700355,0.097488,-0.707106,534.725180,0.830621,55.103302
100,hit,101,5,18,0,-57.909454,77.834539,75.454108,0.579488,-0.405209,-0.707106,536.688133,0.830935,59.719086
104,hit,105,5,20,0,-58.443406,78.740612,104.507108,-0.073733,0.703252,-0.707106,-167.526471,0.949839,86.517334
105,hit,106,5,20,0,-58.443406,78.740612,104.507108,-0.073733,0.703252,-0.707106,663.952405,1.606636,86.517334


In [29]:
sel_hits_ev_df

,hit,#hit,lin_det,piso_det,mód_opt,X',Y',Z',X,Y,Z,tiempo,amp,frec_hits
0,hit,1,5,12,0,-58.891665,77.891023,-11.706892,-0.622119,-0.336108,-0.707106,302.447963,0.610659,72.975159
1,hit,2,5,13,0,-58.955763,78.087135,2.815108,-0.700534,-0.096191,-0.707106,315.962909,0.742721,55.351257
2,hit,3,5,14,0,-58.843884,77.816772,17.343108,-0.563666,-0.426944,-0.707106,340.341512,7.562943,46.081543
3,hit,4,5,15,0,-57.844178,77.956935,31.872108,0.659343,-0.255474,-0.707106,368.401895,3.664037,63.133240
4,hit,5,5,16,0,-58.912526,78.397776,46.383108,-0.647639,0.283838,-0.707106,417.011841,4.868736,0.000000
5,hit,6,5,17,0,-57.810655,78.245451,60.932108,0.700355,0.097488,-0.707106,470.932161,7.625308,55.103302
6,hit,7,5,18,0,-57.909454,77.834539,75.454108,0.579488,-0.405209,-0.707106,536.688133,3.934456,59.719086
7,hit,8,5,20,0,-58.443406,78.740612,104.507108,-0.073733,0.703252,-0.707106,663.952405,3.977190,86.517334


In [74]:
raw_hits_ev_df.loc[raw_hits_ev_df['piso_det'].isin(["13"])]

,hit,#hit,lin_det,piso_det,mód_opt,X',Y',Z',X,Y,Z,tiempo,amp,frec_hits
21,hit,22,5,13,1,-58.028737,77.709229,2.815108,0.433571,-0.558585,-0.707106,315.962909,0.742721,52.127838
22,hit,23,5,13,2,-58.164907,78.700924,2.815108,0.266963,0.654776,-0.707106,-1317.988442,1.028985,52.280426
52,hit,53,3,13,1,-10.100969,36.602132,2.866108,-0.156439,-0.689585,-0.707106,1706.558123,1.115274,58.403015
126,hit,127,1,13,1,4.525052,96.828695,-5.714892,0.205812,0.676492,-0.707106,-547.060613,1.304797,121.116638
127,hit,128,1,13,1,4.525052,96.828695,-5.714892,0.205812,0.676492,-0.707106,2178.366564,1.285060,121.116638


In [71]:
raw_hits_ev_df.loc[raw_hits_ev_df['X\''].isin(["-58.955763"])]

,hit,#hit,lin_det,piso_det,mód_opt,X',Y',Z',X,Y,Z,tiempo,amp,frec_hits


In [73]:
raw_hits_ev_df.loc[raw_hits_ev_df['X\''].isin(["-58.891665"])]

,hit,#hit,lin_det,piso_det,mód_opt,X',Y',Z',X,Y,Z,tiempo,amp,frec_hits
25,hit,26,5,12,0,-58.891665,77.891023,-11.706892,-0.622119,-0.336108,-0.707106,302.447963,0.610659,72.975159


In [ ]:
selp = sel_hits_ev_df.iloc[0]
print("El primer selected pulse es:", selp)
tempdf

In [ ]:
#Aqui justo es donde tengo la duda... ¿que posiciones debo comparar del df? 
#¿Las 8, 9 y 10? ¿ignoro tiempo, amp y frec?

print("Primer selected pulse\n", sel_hits_ev_df.iloc[0])
mod_optico = sel_hits_ev_df.iloc[0][4]

#print("\nMódulo Optico:", mod_optico) #Aqui ya sabemos que es 0 porque es un selected pulse

raw_hits_ev_df.iloc[75]


#### En esta celda yo identifique manualmente y visualmente el que se parece,
##### pero hay que hacerlo de una manera "automatizada", no manual.

## Ahora el código para TODOS los eventos en un mismo archivo (Pendiente)

In [ ]:
#Igual y se requiere un dataframe para guardar la data
hits = pd.DataFrame()

#Leyendo el archivo csv ya sin las líneas no deseadas en un dataframe
file = csv_path+csvarr[0]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None)
#Obteniendo los indices necesarios
ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

for z in range (len(ind_startev)):
    #Definición de variables auxiliares
    difrawhits = ind_bb_selp[0] - ind_bbfit[0]; difselhits = ind_endev[0] - ind_bb_selp[0]
    start_rawhit = ind_bbfit[0] + 1; start_selhit = ind_bb_selp[0]+1
    raw_hits = []; hits_raw_limpios = []; sel_hits = []; sel_hits_limpios = []
    #Obteniendo solo los raw_hits de un evento
    for i in range(difrawhits-1):
        temprawhit = list(data[0][start_rawhit+i].split(" "))
        raw_hits.append(temprawhit)
    #Limpiamos los raw hits obtenidos del evento
    for i in range(len(raw_hits)):
        hits_raw_limpios.append(limpiahit(raw_hits[i]))
    #Obteniendo solo los selected pulses (hits) del mismo evento
    for i in range(difselhits-1):
        tempselhit = list(data[0][start_selhit+i].split(" "))
        sel_hits.append(tempselhit)
    #Limpiamos los raw hits obtenidos del evento
    for i in range(len(sel_hits)):
        sel_hits_limpios.append(limpiahit(sel_hits[i]))
    #Debemos añadir la data a un solo dataframe o lista